### 参考にしたサイト
https://tech.gmogshd.com/transformer/

In [1]:
with open('./data.txt','r',encoding='utf-8') as f:
        text = f.read()
print("テキストの文字数 :", len(text))
print("最初の30文字 : ",text[:30])

テキストの文字数 : 1063
最初の30文字 :  Head Mounted Displayをはじめとした立体視


In [2]:
import torch
import torch.nn as nn

# 使用されている文字
chars = sorted(list(set(text)))
print(chars[:30],chars[50:70])
# 使用されている文字数
char_size = len(chars)

# 文字と数字を一対一対応させる辞書
char2int = { ch : i for i, ch in enumerate(chars) }
int2char = { i : ch for i, ch in enumerate(chars) }

# 文字と数字を変換する関数
encode = lambda a: [char2int[b] for b in a ]
decode = lambda a: ''.join([int2char[b] for b in a ])
print("decode_example:",decode([40,2,5,8,23,56]))

# テキストファイルを数字にして，tensor型に変換
train_data = torch.tensor(encode(text), dtype=torch.long)
print(train_data.shape)
print(train_data[:20])

['\n', ' ', '%', '(', ')', '-', '.', '3', 'A', 'C', 'D', 'F', 'G', 'H', 'L', 'M', 'N', 'P', 'S', 'T', 'U', 'Y', '\\', 'a', 'b', 'c', 'd', 'e', 'g', 'h'] ['か', 'が', 'き', 'く', 'こ', 'さ', 'し', 'じ', 'す', 'そ', 'た', 'っ', 'つ', 'て', 'で', 'と', 'ど', 'な', 'に', 'の']
decode_example: u%-Aaし
torch.Size([1063])
tensor([13, 27, 23, 26,  1, 15, 35, 40, 34, 39, 27, 26,  1, 10, 30, 38, 36, 32,
        23, 43])


In [23]:
class SelfAttention_Head(nn.Module):

    def __init__(self, n_mbed, head_size, block_size):
        super().__init__()
        self.key = nn.Linear(n_mbed, head_size, bias=False)
        self.query = nn.Linear(n_mbed, head_size, bias=False)
        self.value = nn.Linear(n_mbed, head_size, bias=False)
        # 上三角をゼロに，下三角をそのまま
        # 大きいサイズの行列
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))


    ## channelは文字を表現する次元数
    ## Tは文章の長さに相当，足りない部分はpaddingで追加
    ## Bはバッチサイズ，長さが違う文章でもmaskすることで対応している
    def forward(self, x):
        # (Batch_size,data,Channel)
        B, T, C = x.shape
        print(B,T,C)

        k = self.key(x)
        print("k",k)
        q = self.query(x)
        print("q",q)
        v = self.value(x)
        print("v",v)

        #  softmaxの中身計算
        wei = q @ k.transpose(-2,-1)*  (C ** -0.5)
        print(wei)

        # 必要サイズの下三角行列を作成
        # 0に相当する部分を-infで置き換える
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        print(wei)
        
        # 行列の行でsoftmax演算
        wei = nn.functional.softmax(wei, dim=-1)
        print(wei)

        out = wei @ v
        return out

In [22]:
a = torch.tril(torch.ones(4,4))
t = 3
print(a[:t,:t])
print(a[:t,:t]==0)

import numpy as np
A = np.array([[1,2,3],[4,5,6]])
B = np.array([[1,2],[3,4],[5,6]])
C = 2


D = torch.tensor(A @ B * C ** -0.5)
D = nn.functional.softmax(D, dim=-1)


print(D)


E = torch.tril(torch.ones(2,2))
F = D.masked_fill(E == 0,float("-inf"))


print(D)
print(E)
print(F)




# print( A @ B * (C ** -0.5) )
# print( (A @ B * C) ** -0.5)

tensor([[1.4166e-02, 9.8583e-01],
        [2.4751e-05, 9.9998e-01]], dtype=torch.float64)
tensor([[1.4166e-02, 9.8583e-01],
        [2.4751e-05, 9.9998e-01]], dtype=torch.float64)
tensor([[1., 0.],
        [1., 1.]])
tensor([[1.4166e-02,       -inf],
        [2.4751e-05, 9.9998e-01]], dtype=torch.float64)


In [35]:
vector_size = 5

# [単語数] → [単語数，次元数(vector_size)]
embeddings = nn.Embedding(char_size, vector_size)

# e.g. ホログラフィをベクトルにする
encoded_words = torch.tensor(encode("ホログラフィ"))
embeddings_words  = embeddings(encoded_words)
print("[ホログラフィ]のベクトル表現 : \n",embeddings_words)


### 次元を揃える
embeddings_words = embeddings_words.unsqueeze(dim = 0)
print(embeddings_words.shape)

## block_sizeは文章の長さよりも長くする必要がある
attention_head = SelfAttention_Head(n_mbed=vector_size,head_size=3,block_size=embeddings_words.size(1))
attention_head.forward(embeddings_words)

[ホログラフィ]のベクトル表現 : 
 tensor([[ 0.5336, -0.3797, -0.9885,  1.0975, -1.0427],
        [-0.8028, -0.0042, -0.7464, -0.4253,  1.4510],
        [ 2.6586, -0.7181, -1.0346, -0.2757,  0.9967],
        [ 1.2404, -1.5862,  0.3904, -0.0327,  1.3152],
        [-0.9889, -0.1633, -0.4732, -0.8290, -0.4769],
        [-1.3061, -0.0124, -1.1867, -0.3503, -2.6589]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([1, 6, 5])
1 6 5
k tensor([[[-0.2808, -0.0104, -0.1705],
         [-0.0798,  0.1331, -0.3839],
         [-1.2080,  1.4741, -0.1410],
         [-1.0363,  1.1063, -0.9143],
         [ 0.8121, -0.0276,  0.2819],
         [ 1.5220, -0.5073,  0.8169]]], grad_fn=<UnsafeViewBackward0>)
q tensor([[[ 0.1264, -0.4882,  0.7203],
         [ 0.1389,  0.1589, -0.6315],
         [ 1.2587, -0.2302,  0.2275],
         [ 0.2006, -0.2484, -0.2871],
         [ 0.2695,  0.2588,  0.1622],
         [ 0.3580,  0.0847,  1.1614]]], grad_fn=<UnsafeViewBackward0>)
v tensor([[[-0.9900,  0.4442,  0.9319],
         [ 0.4003,

tensor([[[-0.9900,  0.4442,  0.9319],
         [-0.2660, -0.3350, -0.1617],
         [-0.4329, -0.1263, -0.0503],
         [-0.5168,  0.1451,  0.1269],
         [-0.3512,  0.0727, -0.0024],
         [-0.2250,  0.0073,  0.0134]]], grad_fn=<UnsafeViewBackward0>)

In [ ]:
class SelfAttention_MultiHeads(nn.Module):

    def __init__(self, n_mbed, num_heads, head_size, block_size):
        super().__init__()
        self.heads = nn.ModuleList((SelfAttention_Head(n_mbed, head_size, block_size) for _ in range(num_heads)))

    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim = -1)

class FeedForward(nn.Module):

    def __init__(self, n_mbed):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(n_mbed, n_mbed), nn.ReLU())

    def forward(self, x):
        return self.net(x)
